<h1><font color=blue><center>UTILISATION DE MLFLOW</center></font></h1>

MLflow est une plateforme open source permettant de gérer le cycle de vie de l'apprentissage automatique de bout en bout. Il répond à quatre fonctions principales :

Expériences de suivi pour enregistrer et comparer les paramètres et les résultats ( MLflow Tracking ).

Conditionner le code ML sous une forme réutilisable et reproductible afin de le partager avec d'autres data scientists ou de le transférer en production ( MLflow Projects ).

Gérer et déployer des modèles à partir d'une variété de bibliothèques ML vers une variété de plates-formes de service de modèles et d'inférence ( MLflow Models ).

Fournir un magasin de modèles central pour gérer de manière collaborative le cycle de vie complet d'un modèle MLflow, y compris la gestion des versions du modèle, les transitions d'étape et les annotations ( MLflow Model Registry ).

Il faut au préalable avoir un compte ngrok et copié le token que l'on va donné

## Installer mlflow

In [ ]:
! pip install mlflow

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [ ]:
dataset = datasets.load_iris()
dataset

# Création de nos données d'entrainement et de test

In [ ]:
seed = 123
x_train, x_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size= 0.2, random_state=seed)


In [ ]:
print("les dimensions de mon x_train : ",x_train.shape)
print("les dimensions de mon x_test : ",x_test.shape)
print("les dimensions de mon y_train : ",y_train.shape)
print("les dimensions de mon y_test : ",y_test.shape)

# Creation du Model

Dans notre cas on va utiliser du mlflow tracking, vue que l'on va enregistré des expériences en vue d'une comparaison ultérieur

In [ ]:
from sklearn.linear_model import LogisticRegression
import mlflow
mlflow.set_experiment('Reg_logisic_model')
mlflow.sklearn.autolog()
#autolog que l'on viens d'appeler nous permet de faire appel à tous les models de sklearn


Premier run

In [ ]:
with mlflow.start_run(run_name='reg_logistic_para_defaut'):
  clf = LogisticRegression()
  clf.fit(x_train, y_train)

Second run

In [ ]:
with mlflow.start_run(run_name='reg_logistic_para_max_iter=200'):
  clf2 = LogisticRegression(max_iter=200)
  clf2.fit(x_train, y_train)

On va installé le Pyngrok

In [ ]:
! pip install pyngrok

In [ ]:
from pyngrok import ngrok

In [ ]:
ngrok.kill()
# il faut inséré votre token que vous avez préalablement eu après création de votre compte
NGROK_AUT_TOKEN = "2M8hL0PDyX954ZIC3YrNyIX4Y8E_34XBu98He3obmEJQyhdoA"
ngrok.set_auth_token(NGROK_AUT_TOKEN)
# Ouvrir un tunnel HTTPs avec le port d'accès 5000 à travers votre connection localhost
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print ("MLFLOW tracking UI :", ngrok_tunnel.public_url)


In [ ]:
!mlflow ui

[2023-10-28 02:11:26 +0000] [40152] [INFO] Starting gunicorn 21.2.0
[2023-10-28 02:11:26 +0000] [40152] [INFO] Listening at: http://127.0.0.1:5000 (40152)
[2023-10-28 02:11:26 +0000] [40152] [INFO] Using worker: sync
[2023-10-28 02:11:26 +0000] [40157] [INFO] Booting worker with pid: 40157
[2023-10-28 02:11:26 +0000] [40158] [INFO] Booting worker with pid: 40158
[2023-10-28 02:11:26 +0000] [40160] [INFO] Booting worker with pid: 40160
[2023-10-28 02:11:26 +0000] [40159] [INFO] Booting worker with pid: 40159


# On va faire la faire la prediction grâce au code que nous a proposer mlflow

In [ ]:
import numpy as np

In [ ]:
# nouvelle prediction
new_data = np.array([[5.5, 3.8,4.6, 1.9]])
new_data

In [ ]:
import mlflow
logged_model = 'runs:/5fd20da3507c46aabc11c92b0b2cbc01/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
prediction = loaded_model.predict(new_data)
print('votre prediction est :', prediction)

On va créer une nouvelle expérience, dans ce cas on utiliser l'arbre de decision

In [ ]:
from sklearn.tree import DecisionTreeClassifier
mlflow.set_experiment('decision_tree_models')
mlflow.sklearn.autolog()

Premier run

In [ ]:
with mlflow.start_run(run_name='decision_tree_defaut'):
  dt = DecisionTreeClassifier(random_state=seed)
  dt.fit(x_train, y_train)

Second run

In [ ]:
with mlflow.start_run(run_name='decision_tree_max_depht=3'):
  dt2 = DecisionTreeClassifier(random_state=seed, max_depth=3)
  dt2.fit(x_train, y_train)

On va faire une prediction avec celui là aussi pour voir si on aura le même resultat

In [ ]:
import mlflow
logged_model = 'runs:/8f8a057737b64978b487f724a495a6a6/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
prediction = loaded_model.predict(new_data)
print('Votre prediction est :', prediction)